<a href="https://colab.research.google.com/github/appersaravanan/final_project/blob/master/God_grace1_before%20split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from keras.applications import VGG16
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import  MaxPooling2D,UpSampling2D,Input,BatchNormalization,Conv2D,Activation,MaxPool2D,Flatten,Dense,GlobalAveragePooling2D,Lambda,Concatenate,GlobalMaxPooling2D,Add,multiply
from tensorflow.keras.models import Model,Sequential,load_model
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.preprocessing import image as image_utils
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import VGG16
import cv2,os,tempfile
import requests
from keras.preprocessing import image
from scipy.io import loadmat
import tensorflow as tf
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow.keras.layers import  Reshape

In [2]:
img_width, img_height = 224, 224
nb_train_samples = 8144
nb_validation_samples = 4899
epochs = 10
batch_size = 64
n_classes = 196

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content
!mkdir final_project
%cd ./final_project
!mkdir data
%cd data

Mounted at /content/drive
/content
/content/final_project
/content/final_project/data


In [4]:

!rm -f car_devkit.tgz
!wget https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
!tar xvzf car_devkit.tgz

--2020-12-06 16:36:07--  https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 330960 (323K) [application/x-gzip]
Saving to: ‘car_devkit.tgz’

car_devkit.tgz      100%[===================>] 323.20K   258KB/s    in 1.3s    

2020-12-06 16:36:09 (258 KB/s) - ‘car_devkit.tgz’ saved [330960/330960]

devkit/
devkit/cars_meta.mat
devkit/cars_train_annos.mat
devkit/cars_test_annos.mat
devkit/README.txt
devkit/train_perfect_preds.txt
devkit/eval_train.m


In [ ]:
!wget http://imagenet.stanford.edu/internal/car196/cars_train.tgz

!tar xvzf cars_train.tgz
!rm -f cars_train.tgz


In [6]:
%cd /content/final_project/data

/content/final_project/data


In [ ]:
!wget http://imagenet.stanford.edu/internal/car196/cars_test.tgz
!tar xvzf cars_test.tgz
!rm -f cars_test.tgz

In [ ]:
%cd /content/final_project/data/cars_train/
annotsa = loadmat('/content/final_project/data/devkit/cars_meta.mat')
i=1
classes_name=[]
for name in annotsa['class_names'][0]:
  dir_name=str(i)+'_'+str(name[0]).replace(' ','_')
  os.makedirs(dir_name)
  classes_name.append(dir_name)
  i=i+1
classes_name


In [ ]:

annotsa = loadmat('/content/final_project/data/devkit/cars_meta.mat')
i=0
classes_name=[]
for name in annotsa['class_names'][0]:
  dir_name=str(i)+'_'+str(name[0]).replace(' ','_')
  # os.makedirs(dir_name)
  classes_name.append(dir_name)
  i=i+1
classes_name

In [24]:
%cd /content/final_project/data/cars_test/
annotsa = loadmat('/content/final_project/data/devkit/cars_meta.mat')
i=1
classes_name=[]
for name in annotsa['class_names'][0]:
  dir_name=str(i)+'_'+str(name[0]).replace(' ','_')
  os.makedirs(dir_name)
  if dir_name == '174_Ram_C/V_Cargo_Van_Minivan_2012':
    classes_name.append('174_Ram_C')
  else :
    classes_name.append(dir_name)
  i=i+1
classes_name

[Errno 2] No such file or directory: '/content/final_project/data/cars_test/'
/content


FileNotFoundError: ignored

In [12]:
import shutil
from shutil import copyfile
path_base='/content/final_project/data'
train_data_dir = path_base + '/cars_train'
validation_data_dir = path_base + '/cars_test'


annos = loadmat('/content/final_project/data/devkit/cars_train_annos.mat')
data = [[row.flat[0] for row in line] for line in annos['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class', 'fname']
df_train = pd.DataFrame(data, columns=columns)
gp=df_train.groupby('class')
files=""
for index,classes in gp:
    # print (classes_name[index-1])
    files=classes['fname']
    i=0
    for name,file in files.items():
        # print(file)
        copyfile('{}/{}'.format(train_data_dir,file),'{}/{}/{}'.format(train_data_dir,classes_name[index-1],file))
        if i<35:
          copyfile('{}/{}'.format(train_data_dir,file),'{}/{}/{}'.format(validation_data_dir,classes_name[index-1],file))
        i=i+1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
annos = loadmat('/content/final_project/data/devkit/cars_test_annos.mat')
data = [[row.flat[0] for row in line] for line in annos['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class']
df_train = pd.DataFrame(data)
df_train
# gp=df_train.groupby('class')
# files=""
# for index,classes in gp:
#     # print (classes_name[index-1])
#     files=classes['fname']
#     i=0
#     for name,file in files.items():
#         # print(file)
#         # copyfile('{}/{}'.format(train_data_dir,file),'{}/{}/{}'.format(train_data_dir,classes_name[index-1],file))
#         if i<25:
#           copyfile('{}/{}'.format(validation_data_dir,file),'{}/{}/{}'.format(validation_data_dir,classes_name[index-1],file))
#         i=i+1

In [13]:

# train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
#     .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(
    directory=train_data_dir,
    target_size=(img_width, img_height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True
    )

test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)



validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=10,
    class_mode='categorical',
    shuffle=True
    )

Found 8144 images belonging to 196 classes.
Found 6814 images belonging to 196 classes.


In [3]:

vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in enumerate(vgg16_model.layers):
        layer[1].trainable = False
for layer in vgg16_model.layers[15:]:
            layer.trainable = True
#flatten the results from conv block
x = Flatten()(vgg16_model.output)

#add another fully connected layers with batch norm and dropout
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.7)(x)

#add another fully connected layers with batch norm and dropout
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.7)(x)

#add logistic layer with all car classes
predictions = Dense(n_classes, activation='softmax', name='predictions')(x)

# this is the model we will train
model = Model(inputs=vgg16_model.input, outputs=predictions)




In [15]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
# sgd = SGD(lr=0.0001, decay=lr_decay, momentum=0.9, nesterov=True)

In [6]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
callbacks_list = [early_stop, reduce_lr]

In [40]:


earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, mode='auto')
imagenette_builder = tfds.builder("cars196")
imagenette_info = imagenette_builder.info
imagenette_builder.download_and_prepare()
datasets = imagenette_builder.as_dataset(as_supervised=True)
train_data , test_data =datasets['train'],datasets['test']

In [ ]:
def squeeze_excite_block(tensor, ratio=16):
    init = tensor
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)  

    x = multiply([init, se])
    return x

vgg_model = VGG16(include_top=True,weights="imagenet")
vgg_model.trainable =False


#x=squeeze_excite_block(vgg_model.layers[0].output)
x=vgg_model.layers[0].output
x=vgg_model.layers[1](x)
x=vgg_model.layers[2](x)
x=vgg_model.layers[3](x)
x=squeeze_excite_block(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x=vgg_model.layers[4](x)
x=vgg_model.layers[5](x)
x=vgg_model.layers[6](x)
x=squeeze_excite_block(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x=vgg_model.layers[7](x)
x=vgg_model.layers[8](x)
x=vgg_model.layers[9](x)
x=vgg_model.layers[10](x)
x=squeeze_excite_block(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x=vgg_model.layers[11](x)
x=vgg_model.layers[12](x)
x=vgg_model.layers[13](x)
x=vgg_model.layers[14](x)
x=squeeze_excite_block(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x=vgg_model.layers[15](x)
x=vgg_model.layers[16](x)
x=vgg_model.layers[17](x)
x=vgg_model.layers[18](x)
x=squeeze_excite_block(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x=vgg_model.layers[19](x)
x=vgg_model.layers[20](x)
x=vgg_model.layers[21](x)
x = BatchNormalization()(x)
x = Dropout(0.7)(x)
x=Dense(196,activation='softmax')(x)
senet_model = Model(vgg_model.input, x)
     
# senet_model.compile(
#     optimizer='adam',    
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy']
# )

senet_model.summary()
# senet_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
sgd = SGD(lr=0.001, momentum=0.9, nesterov=True)
senet_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, mode='auto')

In [43]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
   
  image,label= (tf.cast(tf.image.resize(image, (224, 224)), tf.float32) / 255.,  tf.one_hot(label,196))
  # image=tf.keras.applications.vgg19.preprocess_input(image)
  return image,label

ds_train = train_data.map(
   normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

ds_train = ds_train.cache()
ds_train = ds_train.shuffle(imagenette_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test= test_data.map(
   normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)
def compareImage(orig,  title,num=10):   
    columns= n = num
    if n>10 :
       columns=n/2;
    plt.figure(figsize=(40, 40))
    print ('\033[1m' + title)
    for i in range(n):
        ax = plt.subplot(2, columns, i+1)
        plt.imshow(orig[i])
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

In [ ]:

ds_test1= test_data.map(
   normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)

t=ds_test1.take(1000)

input_images_array=[]
test1=[]
i=0
for imagedata, label in tfds.as_numpy(t):  # example is (image, label)
  if i>-1:
    input_images_array.append(imagedata) 
    img2 =imagedata.copy()
    test1.append(np.where(label ==1)[0][0])
    print(np.where(label ==1)[0][0])
  i=i+1
print(i)
compareImage(input_images_array,"1",10)

In [44]:
%load_ext tensorboard

In [ ]:

model_history = senet_model.fit(
    ds_train, 
    validation_data=ds_test,     
    shuffle=True,
    epochs=500,
    batch_size=255,
    callbacks=callbacks_list
)



In [12]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


In [ ]:


# from tensorflow.python.keras.utils import data_utils
# import json
# CLASS_INDEX= {
#             '0':  ['n01440764', 'tench'],
#             '1':  ['n02102040', 'English_springer'],
#             '2':  ['n02979186',  'casette_player'],
#             '3':  ['n03000684', 'chain_saw'],
#             '4':  ['n03028079',  'church'],
#             '5':  ['n03394916',  'French_horn'],
#             '6':  ['n03417042',  'garbage_truck'],
#             '7':  ['n03425413',  'gas_pump'],
#             '8':  ['n03445777',  'golf_ball'],
#             '9':  ['n03888257', 'parachute']
#         }
# # CLASS_INDEX_PATH = ('https://storage.googleapis.com/download.tensorflow.org/'
# #                     'data/imagenet_class_index.json')
# # def test_my_decode_predictions(*args, **kwargs):
# #     return my_decode_predictions(*args, **kwargs)


def my_decode_predictions(preds, top=5, **kwargs): 
  
    

    # print(CLASS_INDEX)
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [ ]:
test_images_array=[]
input_images_array=[]
path_base='/content/final_project/data'
train_data_dir = path_base + '/cars_train'
validation_data_dir = path_base + '/cars_test'

t=ds_test1.take(1000)

input_images_array=[]
i=0
for img, label in tfds.as_numpy(t):  # example is (image, label)
  if i>-1:
    input_images_array.append(img) 
    
    # img2 = image_utils.img_to_array(img)  
    pre_processed_image =preprocess_input(img.copy());
    test_images_array.append(img.copy())

  i=i+1
compareImage(input_images_array,"hello",10)

prediction_output_arr=[]
prediction_output_decoded_arr=[]
test2=[]
for i in range(len(test_images_array)): 
  prediction_result= senet_model.predict(test_images_array[i][np.newaxis,...])  
  # prediction_output=my_decode_predictions(prediction_result)
  # prediction_output_arr.append(prediction_result)
  # prediction_output_decoded_arr.append(prediction_output)
  # print(np.where(np.round(prediction_result) ==1))
  if len(np.where(np.round(prediction_result) ==1)[0])>0:
    print(np.where(np.round(prediction_result) ==1)[1][0])
    test2.append(np.where(np.round(prediction_result) ==1)[1][0])
  else:
    compareImage(input_images_array,"hello",10)
    test2.append(-1)
  # compareImage([img],"hello",1)

In [39]:
c=0

for i in range(len(test2)):
  print(test1[i],test2[i],test1[i]==test2[i],CLASS_INDEX[str(test1[i])],CLASS_INDEX[str(test2[i] if test2[i]!=-1 else 0 )] )
  if test1[i]==test2[i]:
    c=c+1
print(c)

141 -1 False ['141', 'Infiniti QX56 SUV 2011'] ['0', 'AM General Hummer SUV 2000']
2 -1 False ['2', 'Acura TL Sedan 2012'] ['0', 'AM General Hummer SUV 2000']
87 165 False ['87', 'Dodge Sprinter Cargo Van 2009'] ['165', 'Mercedes-Benz Sprinter Van 2012']
35 -1 False ['35', 'BMW M6 Convertible 2010'] ['0', 'AM General Hummer SUV 2000']
189 189 True ['189', 'Volkswagen Golf Hatchback 2012'] ['189', 'Volkswagen Golf Hatchback 2012']
111 -1 False ['111', 'Ford GT Coupe 2006'] ['0', 'AM General Hummer SUV 2000']
154 153 False ['154', 'Land Rover LR2 SUV 2012'] ['153', 'Land Rover Range Rover SUV 2012']
78 -1 False ['78', 'Chrysler 300 SRT-8 2010'] ['0', 'AM General Hummer SUV 2000']
98 98 True ['98', 'FIAT 500 Abarth 2012'] ['98', 'FIAT 500 Abarth 2012']
94 -1 False ['94', 'Dodge Durango SUV 2007'] ['0', 'AM General Hummer SUV 2000']
108 -1 False ['108', 'Ford Expedition EL SUV 2009'] ['0', 'AM General Hummer SUV 2000']
140 -1 False ['140', 'Infiniti G Coupe IPL 2012'] ['0', 'AM General Hum

In [ ]:

from shutil import copyfile
path_base='/content/final_project/data'
train_data_dir = path_base + '/cars_train'
validation_data_dir = path_base + '/cars_test'


annos = loadmat('/content/final_project/data/devkit/cars_train_annos.mat')
data = [[row.flat[0] for row in line] for line in annos['annotations'][0]]
columns = ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2', 'class', 'fname']
df_train = pd.DataFrame(data, columns=columns)
df_train

In [ ]:
x=z

In [28]:
CLASS_INDEX={}
for i in range(len(classes_name)):
  CLASS_INDEX[str(i)]=[str(i),classes_name[i]]
  
print(CLASS_INDEX)

{'0': ['0', 'AM General Hummer SUV 2000'], '1': ['1', 'Acura RL Sedan 2012'], '2': ['2', 'Acura TL Sedan 2012'], '3': ['3', 'Acura TL Type-S 2008'], '4': ['4', 'Acura TSX Sedan 2012'], '5': ['5', 'Acura Integra Type R 2001'], '6': ['6', 'Acura ZDX Hatchback 2012'], '7': ['7', 'Aston Martin V8 Vantage Convertible 2012'], '8': ['8', 'Aston Martin V8 Vantage Coupe 2012'], '9': ['9', 'Aston Martin Virage Convertible 2012'], '10': ['10', 'Aston Martin Virage Coupe 2012'], '11': ['11', 'Audi RS 4 Convertible 2008'], '12': ['12', 'Audi A5 Coupe 2012'], '13': ['13', 'Audi TTS Coupe 2012'], '14': ['14', 'Audi R8 Coupe 2012'], '15': ['15', 'Audi V8 Sedan 1994'], '16': ['16', 'Audi 100 Sedan 1994'], '17': ['17', 'Audi 100 Wagon 1994'], '18': ['18', 'Audi TT Hatchback 2011'], '19': ['19', 'Audi S6 Sedan 2011'], '20': ['20', 'Audi S5 Convertible 2012'], '21': ['21', 'Audi S5 Coupe 2012'], '22': ['22', 'Audi S4 Sedan 2012'], '23': ['23', 'Audi S4 Sedan 2007'], '24': ['24', 'Audi TT RS Coupe 2012'], 

In [26]:


classes_name=['AM General Hummer SUV 2000',
             'Acura RL Sedan 2012',
'Acura TL Sedan 2012',
'Acura TL Type-S 2008',
'Acura TSX Sedan 2012',
'Acura Integra Type R 2001',
'Acura ZDX Hatchback 2012',
'Aston Martin V8 Vantage Convertible 2012',
'Aston Martin V8 Vantage Coupe 2012',
'Aston Martin Virage Convertible 2012',
'Aston Martin Virage Coupe 2012',
'Audi RS 4 Convertible 2008',
'Audi A5 Coupe 2012',
'Audi TTS Coupe 2012',
'Audi R8 Coupe 2012',
'Audi V8 Sedan 1994',
'Audi 100 Sedan 1994',
'Audi 100 Wagon 1994',
'Audi TT Hatchback 2011',
'Audi S6 Sedan 2011',
'Audi S5 Convertible 2012',
'Audi S5 Coupe 2012',
'Audi S4 Sedan 2012',
'Audi S4 Sedan 2007',
'Audi TT RS Coupe 2012',
'BMW ActiveHybrid 5 Sedan 2012',
'BMW 1 Series Convertible 2012',
'BMW 1 Series Coupe 2012',
'BMW 3 Series Sedan 2012',
'BMW 3 Series Wagon 2012',
'BMW 6 Series Convertible 2007',
'BMW X5 SUV 2007',
'BMW X6 SUV 2012',
'BMW M3 Coupe 2012',
'BMW M5 Sedan 2010',
'BMW M6 Convertible 2010',
'BMW X3 SUV 2012',
'BMW Z4 Convertible 2012',
'Bentley Continental Supersports Conv. Convertible 2012',
'Bentley Arnage Sedan 2009',
'Bentley Mulsanne Sedan 2011',
'Bentley Continental GT Coupe 2012',
'Bentley Continental GT Coupe 2007',
'Bentley Continental Flying Spur Sedan 2007',
'Bugatti Veyron 16.4 Convertible 2009',
'Bugatti Veyron 16.4 Coupe 2009',
'Buick Regal GS 2012',
'Buick Rainier SUV 2007',
'Buick Verano Sedan 2012',
'Buick Enclave SUV 2012',
'Cadillac CTS-V Sedan 2012',
'Cadillac SRX SUV 2012',
'Cadillac Escalade EXT Crew Cab 2007',
'Chevrolet Silverado 1500 Hybrid Crew Cab 2012',
'Chevrolet Corvette Convertible 2012',
'Chevrolet Corvette ZR1 2012',
'Chevrolet Corvette Ron Fellows Edition Z06 2007',
'Chevrolet Traverse SUV 2012',
'Chevrolet Camaro Convertible 2012',
'Chevrolet HHR SS 2010',
'Chevrolet Impala Sedan 2007',
'Chevrolet Tahoe Hybrid SUV 2012',
'Chevrolet Sonic Sedan 2012',
'Chevrolet Express Cargo Van 2007',
'Chevrolet Avalanche Crew Cab 2012',
'Chevrolet Cobalt SS 2010',
'Chevrolet Malibu Hybrid Sedan 2010',
'Chevrolet TrailBlazer SS 2009',
'Chevrolet Silverado 2500HD Regular Cab 2012',
'Chevrolet Silverado 1500 Classic Extended Cab 2007',
'Chevrolet Express Van 2007',
'Chevrolet Monte Carlo Coupe 2007',
'Chevrolet Malibu Sedan 2007',
'Chevrolet Silverado 1500 Extended Cab 2012',
'Chevrolet Silverado 1500 Regular Cab 2012',
'Chrysler Aspen SUV 2009',
'Chrysler Sebring Convertible 2010',
'Chrysler Town and Country Minivan 2012',
'Chrysler 300 SRT-8 2010',
'Chrysler Crossfire Convertible 2008',
'Chrysler PT Cruiser Convertible 2008',
'Daewoo Nubira Wagon 2002',
'Dodge Caliber Wagon 2012',
'Dodge Caliber Wagon 2007',
'Dodge Caravan Minivan 1997',
'Dodge Ram Pickup 3500 Crew Cab 2010',
'Dodge Ram Pickup 3500 Quad Cab 2009',
'Dodge Sprinter Cargo Van 2009',
'Dodge Journey SUV 2012',
'Dodge Dakota Crew Cab 2010',
'Dodge Dakota Club Cab 2007',
'Dodge Magnum Wagon 2008',
'Dodge Challenger SRT8 2011',
'Dodge Durango SUV 2012',
'Dodge Durango SUV 2007',
'Dodge Charger Sedan 2012',
'Dodge Charger SRT-8 2009',
'Eagle Talon Hatchback 1998',
'FIAT 500 Abarth 2012',
'FIAT 500 Convertible 2012',
'Ferrari FF Coupe 2012',
'Ferrari California Convertible 2012',
'Ferrari 458 Italia Convertible 2012',
'Ferrari 458 Italia Coupe 2012',
'Fisker Karma Sedan 2012',
'Ford F-450 Super Duty Crew Cab 2012',
'Ford Mustang Convertible 2007',
'Ford Freestar Minivan 2007',
'Ford Expedition EL SUV 2009',
'Ford Edge SUV 2012',
'Ford Ranger SuperCab 2011',
'Ford GT Coupe 2006',
'Ford F-150 Regular Cab 2012',
'Ford F-150 Regular Cab 2007',
'Ford Focus Sedan 2007',
'Ford E-Series Wagon Van 2012',
'Ford Fiesta Sedan 2012',
'GMC Terrain SUV 2012',
'GMC Savana Van 2012',
'GMC Yukon Hybrid SUV 2012',
'GMC Acadia SUV 2012',
'GMC Canyon Extended Cab 2012',
'Geo Metro Convertible 1993',
'HUMMER H3T Crew Cab 2010',
'HUMMER H2 SUT Crew Cab 2009',
'Honda Odyssey Minivan 2012',
'Honda Odyssey Minivan 2007',
'Honda Accord Coupe 2012',
'Honda Accord Sedan 2012',
'Hyundai Veloster Hatchback 2012',
'Hyundai Santa Fe SUV 2012',
'Hyundai Tucson SUV 2012',
'Hyundai Veracruz SUV 2012',
'Hyundai Sonata Hybrid Sedan 2012',
'Hyundai Elantra Sedan 2007',
'Hyundai Accent Sedan 2012',
'Hyundai Genesis Sedan 2012',
'Hyundai Sonata Sedan 2012',
'Hyundai Elantra Touring Hatchback 2012',
'Hyundai Azera Sedan 2012',
'Infiniti G Coupe IPL 2012',
'Infiniti QX56 SUV 2011',
'Isuzu Ascender SUV 2008',
'Jaguar XK XKR 2012',
'Jeep Patriot SUV 2012',
'Jeep Wrangler SUV 2012',
'Jeep Liberty SUV 2012',
'Jeep Grand Cherokee SUV 2012',
'Jeep Compass SUV 2012',
'Lamborghini Reventon Coupe 2008',
'Lamborghini Aventador Coupe 2012',
'Lamborghini Gallardo LP 570-4 Superleggera 2012',
'Lamborghini Diablo Coupe 2001',
'Land Rover Range Rover SUV 2012',
'Land Rover LR2 SUV 2012',
'Lincoln Town Car Sedan 2011',
'MINI Cooper Roadster Convertible 2012',
'Maybach Landaulet Convertible 2012',
'Mazda Tribute SUV 2011',
'McLaren MP4-12C Coupe 2012',
'Mercedes-Benz 300-Class Convertible 1993',
'Mercedes-Benz C-Class Sedan 2012',
'Mercedes-Benz SL-Class Coupe 2009',
'Mercedes-Benz E-Class Sedan 2012',
'Mercedes-Benz S-Class Sedan 2012',
'Mercedes-Benz Sprinter Van 2012',
'Mitsubishi Lancer Sedan 2012',
'Nissan Leaf Hatchback 2012',
'Nissan NV Passenger Van 2012',
'Nissan Juke Hatchback 2012',
'Nissan 240SX Coupe 1998',
'Plymouth Neon Coupe 1999',
'Porsche Panamera Sedan 2012',
'Ram C/V Cargo Van Minivan 2012',
'Rolls-Royce Phantom Drophead Coupe Convertible 2012',
'Rolls-Royce Ghost Sedan 2012',
'Rolls-Royce Phantom Sedan 2012',
'Scion xD Hatchback 2012',
'Spyker C8 Convertible 2009',
'Spyker C8 Coupe 2009',
'Suzuki Aerio Sedan 2007',
'Suzuki Kizashi Sedan 2012',
'Suzuki SX4 Hatchback 2012',
'Suzuki SX4 Sedan 2012',
'Tesla Model S Sedan 2012',
'Toyota Sequoia SUV 2012',
'Toyota Camry Sedan 2012',
'Toyota Corolla Sedan 2012',
'Toyota 4Runner SUV 2012',
'Volkswagen Golf Hatchback 2012',
'Volkswagen Golf Hatchback 1991',
'Volkswagen Beetle Hatchback 2012',
'Volvo C30 Hatchback 2012',
'Volvo 240 Sedan 1993',
'Volvo XC90 SUV 2007',
'smart fortwo Convertible 2012']
